# Import necessary libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Initialize Chrome options

In [2]:
chrome_options = Options()

# Set up date formatting for today's date

In [3]:
today = datetime.today().strftime('%Y-%m-%d')

# LinkedIn Credentials and URL of the Linkedin Page I want to scrape

In [13]:
username="joy_ekamel@sacrecoeurgh.edu.eg"
password="joye2015"

page = 'https://www.linkedin.com/search/results/content/?fromMember=%5B%22ACoAAAO3nzQBfuJYkFoVwLugVZkyjyQWPu7WYss%22%5D&heroEntityKey=urn%3Ali%3Afsd_profile%3AACoAAAO3nzQBfuJYkFoVwLugVZkyjyQWPu7WYss&keywords=akash%20agrawal&origin=CLUSTER_EXPANSION&position=0&searchId=70360449-c259-4c24-8181-b4d827fc7a4d&sid=NiU&sortBy=%22date_posted%22'
# if page[-1] == "/":
#     company_name = page.split("/")[-2]
# else:
#     company_name = page.split("/")[-1]
    
# company_name = company_name.replace('-',' ').title()

# Access Webdriver and Open login page

In [5]:
browser = webdriver.Chrome()

browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

# Enter login info

In [6]:
elementID = browser.find_element(By.ID, "username")
elementID.send_keys(username)

elementID = browser.find_element(By.ID, "password")
elementID.send_keys(password)

elementID.submit()

# Go to webpage

In [14]:
#post_page = page + '/posts'
#post_page = post_page.replace('//posts','/posts')
browser.get(page)

# Infinite Scroll

In [15]:
import time

SCROLL_PAUSE_TIME = 1.5
MAX_SCROLLS = False

# JavaScript commands
SCROLL_COMMAND = "window.scrollTo(0, document.body.scrollHeight);"
GET_SCROLL_HEIGHT_COMMAND = "return document.body.scrollHeight"

# Initial scroll height
last_height = browser.execute_script(GET_SCROLL_HEIGHT_COMMAND)
scrolls = 0
no_change_count = 0

while True:
    # Scroll down to bottom
    browser.execute_script(SCROLL_COMMAND)

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script(GET_SCROLL_HEIGHT_COMMAND)
    
    # Increment no change count or reset it
    no_change_count = no_change_count + 1 if new_height == last_height else 0
    
    # Break loop if the scroll height has not changed for 3 cycles or reached the maximum scrolls
    if no_change_count >= 3 or (MAX_SCROLLS and scrolls >= MAX_SCROLLS):
        break
    
    last_height = new_height
    scrolls += 1

## Pressing on all the "see more" buttons

# Enter Beautiful Soup…
We will now collect the source code and then starting looking for the page elements that we want to access.

In [18]:
company_page = browser.page_source
#print(company_page)
linkedin_soup = bs(company_page.encode("utf-8"), "html")

In [21]:
company_name = 'AkashAgrawal'

with open(f"{company_name}_soup.txt", "w+", encoding="utf-8") as t:
    t.write(linkedin_soup.prettify())
#print(linkedin_soup.prettify())

containers = linkedin_soup.find_all("div",{"class":"feed-shared-update-v2"})
containers = [container for container in containers if 'activity' in container.get('data-urn', '')]
print(len(containers))

containers_text = "\n\n".join([c.prettify() for c in containers])
with open(f"{company_name}_soup_containers.txt", "w+",encoding="utf-8" ) as t:
    t.write(containers_text)

50


# Helper functions to help with various tasks
LinkedIn display the post dates as ‘time since posted’ rather than the actual date so this first function called get_actual_date takes in the time period like 5 weeks and determines the approximate date the post was published.

In [22]:
def get_actual_date(date):
    today = datetime.today().strftime('%Y-%m-%d')
    current_year = datetime.today().strftime('%Y')
    
    def get_past_date(days=0, weeks=0, months=0, years=0):
        date_format = '%Y-%m-%d'
        dtObj = datetime.strptime(today, date_format)
        past_date = dtObj - relativedelta(days=days, weeks=weeks, months=months, years=years)
        past_date_str = past_date.strftime(date_format)
        return past_date_str

    past_date = date
    
    if 'hour' in date:
        past_date = today
    elif 'day' in date:
        date.split(" ")[0]
        past_date = get_past_date(days=int(date.split(" ")[0]))
    elif 'week' in date:
        past_date = get_past_date(weeks=int(date.split(" ")[0]))
    elif 'month' in date:
        past_date = get_past_date(months=int(date.split(" ")[0]))
    elif 'year' in date:
        past_date = get_past_date(months=int(date.split(" ")[0]))
    else:
        split_date = date.split("-")
        if len(split_date) == 2:
            past_month = split_date[0]
            past_day =  split_date[1]
            if len(past_month) < 2:
                past_month = "0"+past_month
            if len(past_day) < 2:
                past_day = "0"+past_day
            past_date = f"{current_year}-{past_month}-{past_day}"
        elif len(split_date) == 3:
            past_month = split_date[0]
            past_day =  split_date[1]
            past_year = split_date[2]
            if len(past_month) < 2:
                past_month = "0"+past_month
            if len(past_day) < 2:
                past_day = "0"+past_day
            past_date = f"{past_year}-{past_month}-{past_day}"

    return past_date

def convert_abbreviated_to_number(s):
    if 'K' in s:
        return int(float(s.replace('K', '')) * 1000)
    elif 'M' in s:
        return int(float(s.replace('M', '')) * 1000000)
    else:
        return int(s)

# Define a data structure to hold all the post information

In [23]:
import re

posts_data = []

def get_text(container, selector, attributes):
    try:
        element = container.find(selector, attributes)
        if element:
            return element.text.strip()
    except Exception as e:
        print(e)
    return ""

# Function to extract media information
def get_media_info(container):
    media_info = [("div", {"class": "update-components-video"}, "Video"),
                  ("div", {"class": "update-components-linkedin-video"}, "Video"),
                  ("div", {"class": "update-components-image"}, "Image"),
                  ("article", {"class": "update-components-article"}, "Article"),
                  ("div", {"class": "feed-shared-external-video__meta"}, "Youtube Video"),
                  ("div", {"class": "feed-shared-mini-update-v2 feed-shared-update-v2__update-content-wrapper artdeco-card"}, "Shared Post"),
                  ("div", {"class": "feed-shared-poll ember-view"}, "Other: Poll, Shared Post, etc")]
    
    for selector, attrs, media_type in media_info:
        element = container.find(selector, attrs)
        if element:
            link = element.find('a', href=True)
            return link['href'] if link else "None", media_type
    return "None", "Unknown"

# Main loop to process each container
We will now loop through every post on the page, using the element identifiers that we found in the previous step to collect the data that we want and add to our list of post_data. This is the part of the code that could potentially need updating by the time you read this, but it will usually be a slight change in class names or something similar.

In [34]:
for container in containers:
    post_text = get_text(container, "div", {"class": "feed-shared-update-v2__description-wrapper"})
    media_link, media_type = get_media_info(container)
    post_date = get_text(container, "div", {"class": "update-components-text-view break-words"})
    post_date = get_actual_date(post_date)
    
    # Reactions (likes)
    reactions_element = container.find_all(lambda tag: tag.name == 'button' and 'aria-label' in tag.attrs and 'reaction' in tag['aria-label'].lower())
    reactions_idx = 1 if len(reactions_element) > 1 else 0
    post_reactions = reactions_element[reactions_idx].text.strip() if reactions_element and reactions_element[reactions_idx].text.strip() != '' else 0
    print(post_reactions)
    
    # Comments
    comment_element = container.find_all(lambda tag: tag.name == 'button' and 'aria-label' in tag.attrs and 'comment' in tag['aria-label'].lower())
    comment_idx = 1 if len(comment_element) > 1 else 0
    post_comments = comment_element[comment_idx].text.strip() if comment_element and comment_element[comment_idx].text.strip() != '' else 0

    # Shares
    shares_element = container.find_all(lambda tag: tag.name == 'button' and 'aria-label' in tag.attrs and 'repost' in tag['aria-label'].lower())
    shares_idx = 1 if len(shares_element) > 1 else 0
    post_shares = shares_element[shares_idx].text.strip() if shares_element and shares_element[shares_idx].text.strip() != '' else 0
        
        
    # Append the collected data to the posts_data list
    posts_data.append({
        "Page": company_name,
        "Date": post_date,
        "Post Text": post_text,
        "Likes": post_reactions,
        "Shares":post_shares,
        #"Likes Numeric": convert_abbreviated_to_number(post_reactions),
    })

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [35]:
try:
    df = pd.DataFrame(posts_data)
except:
    for item in list(data.keys()):
        print(item)
        print(len(data[item]))
        
for col in df.columns:
    try:
        df[col] = df[col].astype(int)
    except:
        pass
    
df.sort_values(by="Likes", inplace=True, ascending=False)
df

,Page,Date,Post Text,Media Type,Likes,Comments,Shares,Media Link
0,AkashAgrawal,,We almost shut shop back in 2020. . When we be...,Image,0,Comment,3 reposts,None
263,AkashAgrawal,3mo • \n\n 3mo •,Receiving acknowledgment from someone who has ...,NaN,0,NaN,0,NaN
273,AkashAgrawal,5mo • \n\n 5mo •,"Masale, Mithai, aur Mubarak! 🎇 This festive se...",NaN,0,NaN,1 repost,NaN
272,AkashAgrawal,5mo • \n\n 5mo •,20% of all advertising features a celebrity an...,NaN,0,NaN,1 repost,NaN
271,AkashAgrawal,5mo • \n\n 5mo •,"Chaahe english gaano pe bhangra ki baat ho, ya...",NaN,0,NaN,0,NaN
...,...,...,...,...,...,...,...,...
130,AkashAgrawal,,"Back in 2014, we barely had 350 startups in th...",NaN,0,NaN,2 reposts,NaN
129,AkashAgrawal,,"Kuch ho na ho, par khana laajawab hona chahiye...",NaN,0,NaN,2 reposts,NaN
128,AkashAgrawal,,The food of the past is now becoming food of t...,NaN,0,NaN,0,NaN
127,AkashAgrawal,,1 in 4 entrepreneurs face a mental health diso...,NaN,0,NaN,0,NaN


Lastly, we turn our post_data list which contains individual dictionaries of post data and create a pandas DataFame, and finally export it as a CSV and/or Excel file.

In [36]:
df.to_csv("{}_posts.csv".format(company_name), encoding='utf-8', index=False)
# writer = pd.ExcelWriter("{}_linkedin_posts.xlsx".format(company_name), engine='xlsxwriter')
# df.to_excel(writer, index = False)
# writer.save()